In [1]:
import os
os.chdir("/usr/src/app")
import pandas as pd
from IPython.display import display, clear_output
import time
import re
import numpy as np

In [2]:
from deepsetstats.paths import (
    PATH_MASTER_GS,
    PATH_VIDEOS
)

In [3]:
# Path dataframe
df = pd.read_parquet(PATH_MASTER_GS, engine="pyarrow")

# Prepare folder

In [4]:
import os

In [5]:
PATH_OUT_VIDEOS = PATH_VIDEOS
EXTENSION = ".mp4"

# Temporal directory to save downloaded videos and final directory
os.makedirs(PATH_OUT_VIDEOS, exist_ok=True)

# Download video


In [6]:
import yt_dlp as youtube_dl

def download_video(video_id, path_out=PATH_OUT_VIDEOS):

    ydl_opts = {
        "format": "bestvideo[ext=mp4]/best[ext=mp4]",  # Download the best quality available
        "outtmpl": f"{PATH_OUT_VIDEOS}/{video_id}.%(ext)s",  # Specify the output template
    }
    
    # Create the downloader
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([f"https://www.youtube.com/watch?v={video_id}"])

    return True

# Iterate on videos


In [17]:
PATH_OUT_VIDEOS

'videos/all'

In [18]:
l_videos_downloaded = os.listdir(PATH_OUT_VIDEOS)

# Set of already downloaded videos
s_videos_downloaded = set()

for vid in l_videos_downloaded:
    if vid.endswith(EXTENSION):
        vid_id = vid.split(EXTENSION)[0]
        s_videos_downloaded.add(vid_id)

In [19]:
d_success = {
    vid_id: True for vid_id in list(s_videos_downloaded)
}

### Patch

Only download videos from a given set of tournaments

In [21]:
s_tournaments_id_download = {0, 1, 2}

In [22]:
# Add a column of the already downloaded videos
df["downloaded"] = np.where(df["video_id"].isin(s_videos_downloaded), True, False)

In [23]:
# Select not-downloaded videos
df_pending = df[(~df["downloaded"]) & (df["tournament_id"].isin(s_tournaments_id_download))].copy()
df_pending["year"] = df_pending["year"].astype(int)

In [24]:
# Step 1: Sort the DataFrame by 'group' and 'date'
df_pending = df_pending.sort_values(by=['tournament_id', 'year'], ascending=[False, False])

# Create a number for each row within that tournament_id
df_pending["group_val"] = df_pending.groupby('tournament_id').cumcount() + 1

# Sort by that number
df_pending = df_pending.sort_values(by='group_val', ascending=True).drop(columns='group_val')

In [25]:
df_pending.head(20)

,video_id,player_id,tournament_id,tournament_name,name,title,year,downloaded
2836,LVwGp_WFcwc,2331,2,Wimbledon,Marcel Granollers,Mektic/Pavic vs Granollers/Zeballos | Mens' D...,2021,False
1758,pDRdgeV2yjc,13,0,Australian Open,Tommy Paul,Ben Shelton v Tommy Paul Extended Highlights |...,2023,False
1115,77ksb2e8cKE,4,1,Roland Garros,Stefanos Tsitsipas,Stefanos Tsitsipas vs Pablo Carreno Busta - Ro...,2021,False
2837,9Or0anXGNvk,1,2,Wimbledon,Novak Djokovic,Novak Djokovic vs Denis Shapovalov | Semi-Fina...,2021,False
1116,77ksb2e8cKE,22,1,Roland Garros,Pablo Carreno Busta,Stefanos Tsitsipas vs Pablo Carreno Busta - Ro...,2021,False
1759,pDRdgeV2yjc,42,0,Australian Open,Ben Shelton,Ben Shelton v Tommy Paul Extended Highlights |...,2023,False
2838,9Or0anXGNvk,21,2,Wimbledon,Denis Shapovalov,Novak Djokovic vs Denis Shapovalov | Semi-Fina...,2021,False
1760,i9OExTiiMsE,1,0,Australian Open,Novak Djokovic,Andrey Rublev v Novak Djokovic Extended Highli...,2023,False
1117,EB5MgKv17xU,2059,1,Roland Garros,Roger Federer,Roger Federer vs Dominik Köpfer - Round 3 High...,2021,False
1118,sqJXFPkTSMM,17,1,Roland Garros,Lorenzo Musetti,Lorenzo Musetti vs Marco Cecchinato - Round 3 ...,2021,False


In [ ]:
d_failures = {} 
for i, row in df_pending.iterrows():        
    vid_id = row["video_id"]

    # If video already downloaded, do not download it again
    if vid_id in d_success:
        continue
    if vid_id in s_videos_downloaded:
        continue

    if i in [0, 50, 100, 150, 200]:
        print("--" * 30)
        print(f"Iteration: {i}")
        print("--" * 30)
    elif (i % 50) == 0:
        print("--" * 30)
        print(f"Iteration: {i}")
        print("--" * 30)
        #print("Sleeping")
        #time.sleep(3000)

    # Download video
    print("--" * 30)
    print("DOWNLOADING VIDEO:", vid_id)
    print("--" * 30)
    try:
        tt = download_video(vid_id)
    except Exception as e:
        d_failures[vid_id] = str(e)
        continue
    if tt:
        d_success[vid_id] = True

------------------------------------------------------------
DOWNLOADING VIDEO: LVwGp_WFcwc
------------------------------------------------------------
[youtube] Extracting URL: https://www.youtube.com/watch?v=LVwGp_WFcwc
[youtube] LVwGp_WFcwc: Downloading webpage


[youtube] LVwGp_WFcwc: Downloading ios player API JSON
[youtube] LVwGp_WFcwc: Downloading android player API JSON
[youtube] LVwGp_WFcwc: Downloading iframe API JS
[youtube] LVwGp_WFcwc: Downloading player 019a2dc2
[youtube] LVwGp_WFcwc: Downloading web player API JSON


ERROR: [youtube] LVwGp_WFcwc: The uploader has not made this video available in your country


------------------------------------------------------------
DOWNLOADING VIDEO: pDRdgeV2yjc
------------------------------------------------------------
[youtube] Extracting URL: https://www.youtube.com/watch?v=pDRdgeV2yjc
[youtube] pDRdgeV2yjc: Downloading webpage


[youtube] pDRdgeV2yjc: Downloading ios player API JSON
[youtube] pDRdgeV2yjc: Downloading android player API JSON
[youtube] pDRdgeV2yjc: Downloading iframe API JS
[youtube] pDRdgeV2yjc: Downloading player 019a2dc2
[youtube] pDRdgeV2yjc: Downloading web player API JSON
[youtube] pDRdgeV2yjc: Downloading m3u8 information


...

[youtube] MpOgFy18q_c: Downloading initial data API JSON
[info] MpOgFy18q_c: Downloading 1 format(s): 616
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 60
[download] Destination: videos/all/MpOgFy18q_c.mp4
[download] 100% of  151.47MiB in 00:01:38 at 1.54MiB/s                     
------------------------------------------------------------
DOWNLOADING VIDEO: smk2IuKycnc
------------------------------------------------------------
[youtube] Extracting URL: https://www.youtube.com/watch?v=smk2IuKycnc
[youtube] smk2IuKycnc: Downloading webpage


[youtube] smk2IuKycnc: Downloading ios player API JSON
[youtube] smk2IuKycnc: Downloading android player API JSON
[youtube] smk2IuKycnc: Downloading iframe API JS
[youtube] smk2IuKycnc: Downloading player 019a2dc2
[youtube] smk2IuKycnc: Downloading web player API JSON
[youtube] smk2IuKycnc: Downloading m3u8 information


[youtube] smk2IuKycnc: Downloading initial data API JSON
[info] smk2IuKycnc: Downloading 1 format(s): 612
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1596
[download] Destination: videos/all/smk2IuKycnc.mp4
[download]  14.7% of ~   1.57GiB at    9.69KiB/s ETA 40:20:58 (frag 233/1596)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[download]  66.0% of ~   1.58GiB at    3.48KiB/s ETA 44:49:41 (frag 1053/1596)

# See video


In [15]:
import cv2
import random
import matplotlib.pyplot as plt

In [16]:
video_id = "RS_4HosMJuE"

# Replace with the actual path to your MP4 video file
video_path = os.path.join(PATH_OUT_VIDEOS, f"{video_id}.mp4")

# Open the video capture object
cap = cv2.VideoCapture(video_path)

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Generate a random frame index
random_frame_index = random.randint(0, total_frames - 1)

# Set the frame position to the random index
cap.set(cv2.CAP_PROP_POS_FRAMES, random_frame_index)

# Read the frame at the random index
ret, frame = cap.read()

# Check if the frame was read successfully
if not ret:
    print("Error: Could not read frame.")
    cap.release()
    exit()

In [ ]:
# Convert the frame from BGR to RGB
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Plot the first frame using matplotlib
plt.imshow(frame_rgb)
plt.axis("off")  # Turn off axis labels and ticks
plt.title("First Frame")
plt.show()